In [0]:
# Build dim_date from daily, monthly, and quarterly datasets

import pandas as pd
from pyspark.sql.functions import (
    col, year, month, dayofmonth, dayofweek, weekofyear,
    quarter, dayofyear, date_format, lit
)

# === Load from mounted Delta tables ===
df_daily = spark.read.format("delta").load("/mnt/fred/daily").select("date")
df_monthly = spark.read.format("delta").load("/mnt/fred/monthly").select("date")
df_quarterly = spark.read.format("delta").load("/mnt/fred/quarterly").select("date")

# === Union and deduplicate ===
dim_date = df_daily.union(df_monthly).union(df_quarterly).distinct()

# === Add calendar attributes ===
dim_date = dim_date \
    .withColumn("year", year("date")) \
    .withColumn("quarter", quarter("date")) \
    .withColumn("month", month("date")) \
    .withColumn("day", dayofmonth("date")) \
    .withColumn("day_of_week", dayofweek("date")) \
    .withColumn("week_of_year", weekofyear("date")) \
    .withColumn("day_of_year", dayofyear("date")) \
    .withColumn("is_weekend", (dayofweek("date").isin([1, 7])).cast("boolean")) \
    .withColumn("month_name", date_format("date", "MMMM")) \
    .withColumn("day_name", date_format("date", "EEEE")) \
    .withColumn("is_start_of_month", (dayofmonth("date") == 1).cast("boolean"))

# === Save dim_date to mounted storage in Delta format ===
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")
dim_date.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save("/mnt/fred/dim_date")  